In [2]:
from __future__ import absolute_import, print_function, division
import codecs
import glob
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/suraj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/suraj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
book_filenames = sorted(glob.glob('/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/*.txt'))
print(book_filenames)

["/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 1 - The Philosopher's Stone_djvu.txt", '/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 2 - The Chamber of Secrets_djvu.txt', '/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 3 - The Prisoner of Azkaban_djvu.txt', '/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 4 - The Goblet of Fire_djvu.txt', '/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 5 - The Order of the Phoenix_djvu.txt', '/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 6 - The Half Blood Prince_djvu.txt', '/home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 7 - The Deathly Hallows_djvu.txt']


In [5]:
raw_corpus = u''

for file in book_filenames:
    print("Reading in book {0}".format(file))
    with codecs.open(file, 'r', encoding='utf-8') as book_file:
        raw_corpus += book_file.read()
    print("Added book to corpus, Corpus Length: {0}".format(len(raw_corpus)))

Reading in book /home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 1 - The Philosopher's Stone_djvu.txt
Added book to corpus, Corpus Length: 474429
Reading in book /home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 2 - The Chamber of Secrets_djvu.txt
Added book to corpus, Corpus Length: 1006137
Reading in book /home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 3 - The Prisoner of Azkaban_djvu.txt
Added book to corpus, Corpus Length: 1683115
Reading in book /home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 4 - The Goblet of Fire_djvu.txt
Added book to corpus, Corpus Length: 2870365
Reading in book /home/suraj/Repositories/Deep Learning/Word Vectors/Harry Potter word vectors/books/Book 5 - The Order of the Phoenix_djvu.txt
Added book to corpus, Corpus Length: 4479128
Reading in book /home/suraj/Repositories/Deep Learning/Word Vectors/Har

In [6]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [7]:
raw_sentences = tokenizer.tokenize(raw_corpus)

In [8]:
# Clean sentences and convert to words
def sentence_to_words(sentence):
    """
    
    :param sentence: Input sentences 
    :return: Return a list of words for the sentence
    """
    cleaned_sentence = re.sub("[^a-zA-Z]", " ", sentence)
    words = cleaned_sentence.split()
    return words

In [14]:
print(len(raw_sentences))

63914


In [20]:
print(raw_sentences[63911])
print(sentence_to_words(raw_sentences[63911]))

“I know he will.” 

The scar had not pained Harry for nineteen years.
[u'I', u'know', u'he', u'will', u'The', u'scar', u'had', u'not', u'pained', u'Harry', u'for', u'nineteen', u'years']


In [22]:
sentence = []
for raw_sentence in raw_sentences:
    if len(raw_sentences) > 0:
        sentence.append(sentence_to_words(raw_sentence))

In [29]:
# Print the first 10 tokenized sentences
print(sentence[0:10])

[[u'THE', u'BOY', u'WHO', u'LIVED', u'Mr', u'and', u'Mrs', u'Dursley', u'of', u'number', u'four', u'Privet', u'Drive', u'were', u'proud', u'to', u'say', u'that', u'they', u'were', u'perfectly', u'normal', u'thank', u'you', u'very', u'much'], [u'They', u'were', u'the', u'last', u'people', u'you', u'd', u'expect', u'to', u'be', u'involved', u'in', u'anything', u'strange', u'or', u'mysterious', u'because', u'they', u'just', u'didn', u't', u'hold', u'with', u'such', u'nonsense'], [u'Mr', u'Dursley', u'was', u'the', u'director', u'of', u'a', u'firm', u'called', u'Grunnings', u'which', u'made', u'drills'], [u'He', u'was', u'a', u'big', u'beefy', u'man', u'with', u'hardly', u'any', u'neck', u'although', u'he', u'did', u'have', u'a', u'very', u'large', u'mustache'], [u'Mrs', u'Dursley', u'was', u'thin', u'and', u'blonde', u'and', u'had', u'nearly', u'twice', u'the', u'usual', u'amount', u'of', u'neck', u'which', u'came', u'in', u'very', u'useful', u'as', u'she', u'spent', u'so', u'much', u'of'

In [32]:
# Get total number of tokens
tokens_len = sum([len(s) for s in sentence])
print("The Harry Potter Book Corpus has {0:,} tokens ".format(tokens_len))

The Harry Potter Book Corpus has 1,174,677 tokens 
